In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_auc_score, confusion_matrix, average_precision_score
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from time import time
import itertools, gc, operator, os, pickle
import multiprocessing as mp
import xgboost as xgb

/opt/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
label_name = 'mt_hh'
os.system('mkdir '+label_name)

paths = ['../../../data/pnc/inbound/clv_hhsamp'+n+'.csv' for n in list('1234')]

## Getting eligible data 

In [3]:
def sel_sample(path, feature, values, chunksize, usecols=None):
    reader = pd.read_csv(path, chunksize=chunksize, usecols=usecols)
    data = None
    start = time()
    for j, chunk in enumerate(reader):
        items = chunk.loc[chunk[feature].isin(values)]
        data = pd.concat([data, items])
        del items
        if j%10 == 9:
            print('{} seconds: completed {} rows'.format(round(time() - start,2), (j+1)*chunksize))
    gc.collect()
    return data

In [4]:
def get_y_q3(paths, label_name, sel_by, sv_key, sv_label):
    start = time()

    parallel = mp.Pool(processes=len(paths)) 
    returner = [parallel.apply_async(sel_sample, args=(paths[i],), 
                kwds={'feature':sel_by, 'values':sv_label+[sv_key], 'chunksize':100000, 'usecols':['rlb_location_key', label_name, sel_by]})
                for i in range(len(paths))]

    print('Begin extracting data ...')
    items = pd.concat([p.get() for p in returner])
    parallel.close()
    parallel.join()
    del parallel
    gc.collect()
    
    keys = items.loc[(items[label_name]==0) & (items[sel_by]==sv_key), ['rlb_location_key']]
    data_y = items.groupby('rlb_location_key')[label_name].max().reset_index()
    data_y = pd.merge(keys, data_y, on='rlb_location_key', how='left')
    print('----------')
    print('The overall processing time is {} seconds.'.format(round(time() - start,2)))
    del keys, items
    return data_y

In [5]:
def get_X_q3(paths, label_name, feature, values):
    file_paths = []
    for i, path in enumerate(paths):
        print('====================')
        filename = path[-1*path[::-1].find('/'):] if path.find('/') != -1 else path
        print('Begin processing data from '+filename+' ......')
        items = sel_sample(path, feature, values, chunksize=100000, usecols=None)
        print('----- Done! Begin selecting eligible samples ...')
        group = items.groupby('rlb_location_key')[label_name].count().reset_index()
        keys = group.loc[group[label_name]==len(values), 'rlb_location_key'].values
        items = items.loc[items.rlb_location_key.isin(keys)].sort_values(['rlb_location_key',feature]).reset_index(drop=True)
        print('----- Done!')
        save_path = label_name+'/data_q3_X_'+str(i+1)+'.csv'
        items.to_csv(save_path, index=False)
        print('The file has been saved to '+save_path)
        file_paths.append(save_path)
        del items
        gc.collect()
    print(str(len(paths))+' files have been save!')
    return file_paths

In [6]:
data_y = get_y_q3(paths, label_name, 'time_period', sv_key='2016-10-31', 
                  sv_label=['2016-11-30','2016-12-31','2017-01-31','2017-02-28','2017-03-31','2017-04-30'])
data_y.to_csv(label_name+'/data_q3_y.csv', index=False)

Begin extracting data ...
26.85 seconds: completed 1000000 rows
26.91 seconds: completed 1000000 rows
27.13 seconds: completed 1000000 rows
27.27 seconds: completed 1000000 rows
55.61 seconds: completed 2000000 rows
55.68 seconds: completed 2000000 rows
55.81 seconds: completed 2000000 rows
57.28 seconds: completed 2000000 rows
84.1 seconds: completed 3000000 rows
84.24 seconds: completed 3000000 rows
84.72 seconds: completed 3000000 rows
85.97 seconds: completed 3000000 rows
110.16 seconds: completed 4000000 rows
110.17 seconds: completed 4000000 rows
110.73 seconds: completed 4000000 rows
112.67 seconds: completed 4000000 rows
141.86 seconds: completed 5000000 rows
141.92 seconds: completed 5000000 rows
142.26 seconds: completed 5000000 rows
143.2 seconds: completed 5000000 rows
160.23 seconds: completed 6000000 rows
160.26 seconds: completed 6000000 rows
160.32 seconds: completed 6000000 rows
160.46 seconds: completed 6000000 rows
----------
The overall processing time is 161.81 sec

In [7]:
file_paths = get_X_q3(paths, label_name, 'time_period',  
                      ['2015-09-30','2015-10-31','2015-11-30','2015-12-31','2016-01-31','2016-02-29',
                       '2016-03-31','2016-04-30','2016-05-31','2016-06-30','2016-07-31','2016-08-31'])

Begin processing data from clv_hhsamp1.csv ......
41.19 seconds: completed 1000000 rows
90.07 seconds: completed 2000000 rows
146.42 seconds: completed 3000000 rows
213.85 seconds: completed 4000000 rows


/opt/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


287.91 seconds: completed 5000000 rows
370.48 seconds: completed 6000000 rows
----- Done! Begin selecting eligible samples ...
----- Done!
The file has been saved to mt_hh/data_q3_X_1.csv
Begin processing data from clv_hhsamp2.csv ......
42.72 seconds: completed 1000000 rows
100.94 seconds: completed 2000000 rows
165.55 seconds: completed 3000000 rows
234.72 seconds: completed 4000000 rows
312.96 seconds: completed 5000000 rows
402.24 seconds: completed 6000000 rows
----- Done! Begin selecting eligible samples ...
----- Done!
The file has been saved to mt_hh/data_q3_X_2.csv
Begin processing data from clv_hhsamp3.csv ......
47.67 seconds: completed 1000000 rows
117.74 seconds: completed 2000000 rows
182.63 seconds: completed 3000000 rows
260.62 seconds: completed 4000000 rows
348.31 seconds: completed 5000000 rows
451.64 seconds: completed 6000000 rows
----- Done! Begin selecting eligible samples ...
----- Done!
The file has been saved to mt_hh/data_q3_X_3.csv
Begin processing data from

In [3]:
data_y = pd.read_csv(label_name+'/data_q3_y.csv')
file_paths = [label_name+'/data_q3_X_'+str(n+1)+'.csv' for n in range(4)]
data_X = None
for path in file_paths:
    temp = pd.read_csv(path)
    temp = pd.merge(temp.drop(label_name, axis=1), data_y, on='rlb_location_key', how='inner')
    data_X = pd.concat([data_X, temp])
    del temp
    gc.collect()

/opt/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Pre-processing

In [5]:
col_drop = [
            'rel_tenure_src',
            'dd_agr_type',
            'sv_agr_type',
            'mm_agr_type',
            'cd_agr_type',
            'cd_ira_agr_type',
            'bk_agr_type',
            'bk_ira_agr_type',
            'ir_agr_type',
            'pp_agr_type',
            'cc_agr_type',
            'mt_agr_type',
            'heil_agr_type',
            'heloc_agr_type',
            'pil_agr_type',
            'ploc_agr_type',
            'auto_agr_type',
            'sl_agr_type',
            'iil_agr_type',
            'sd_agr_type',
            'in_agr_type',
            'psycle_code_ne',
            'sls_branch_book',
            'market_book',
            'zip_code',
            'market_zip',
            'rcb_consumer_hh',
            'total_profit',
            'total_profit_var',
            'total_rev',
            'total_net_int_inc',
            'total_non_int_inc',
            'total_exp',
            'total_exp_fix',
            'total_exp_var',
            'total_exp_dis',
            'mt_acct',
            'mt_bal',
            'mt_int',
            'mt_rev',
            'mt_exp',
            'mt_tenure',
            'mt_conv_hh',
            'mt_start_bal',
            'ixiwc_total_assets',
            'ixiwc_inv',
            'ixiwc_deposits',
            'ixiwc_dd',
            'ixiwc_sv',
            'ixiwc_mm',
            'ixiwc_cd',
            'fico_auto',
            'fico_cc',
            'fico_heil',
            'fico_heloc',
            'fico_mt',
            'date_opened_first_prod']

data_X.drop(col_drop, axis=1, inplace=True)

In [7]:
dma_entry = list(data_X.dma.value_counts().index[data_X.dma.value_counts()<=40000])
data_X.loc[data_X.dma.isin(dma_entry),'dma'] = 'Garbage'
data_X.dma.value_counts()

Garbage                    864775
Philadelphia, PA           386080
Pittsburgh, PA             317274
New York, NY               287684
Washington et al, DC-MD    218109
Chicago, IL                206681
Cleveland et al, OH        203142
Detroit, MI                156328
Cincinnati, OH             117054
Indianapolis, IN           115728
Baltimore, MD              114889
Louisville, KY             111046
Columbus, OH               106201
Harrisburg et al, PA        76648
Wilkes Barre et al, PA      74835
W. Palm Beach et al, FL     69853
Grand Rapids et al, MI      63105
Raleigh et al, NC           62523
Atlanta, GA                 55584
Dayton, OH                  48997
St. Louis, MO               46713
Orlando et al, FL           46311
Name: dma, dtype: int64

In [8]:
col_cat = ['inc_code_hh',
           'hh_agr_type',
           'core_agr_type',
           'new_hh_traj_acq',
           'new_hh_traj_seg',
           'sales_channel',
           'age_grp_4L',
           'age_hh_src',
           'inc_grp_3L',
           'inc_code_hh_src',
           'consumer_segment',
           'lifestage',
           'dma',
           'market_clv']
for f in col_cat:
    if sum(data_X[f].isnull()) > 0:
        data_X.loc[data_X[f].isnull(),f] = 'missing'

In [10]:
data_X = pd.get_dummies(data_X, columns=col_cat, drop_first=True)

In [15]:
y = data_X.loc[:,['rlb_location_key', label_name]]
data_X.drop(label_name, axis=1, inplace=True)

In [18]:
data_X.shape, y.shape

((3761532, 328), (3761532, 2))

### Splitting training/test

In [19]:
data_X.iloc[:2640000,:].to_csv(label_name+'/data_q3_X_trn.csv', index=False)
data_X.iloc[2640000:,:].to_csv(label_name+'/data_q3_X_tst.csv', index=False)

In [21]:
y.iloc[:2640000,:].to_csv(label_name+'/data_q3_y_trn.csv', index=False)
y.iloc[2640000:,:].to_csv(label_name+'/data_q3_y_tst.csv', index=False)

### Reloading saved data

In [5]:
X_trn = pd.read_csv(label_name+'/data_q3_X_trn.csv')

In [6]:
X_trn.drop(['rlb_location_key','time_period'], axis=1, inplace=True)

In [3]:
X_tst = pd.read_csv(label_name+'/data_q3_X_tst.csv')

In [8]:
X_tst.drop(['rlb_location_key','time_period'], axis=1, inplace=True)

In [4]:
X_tst.shape

(1121532, 328)

In [5]:
X_trn.shape

(2640000, 326)

Impute the missing using the most frequent value

In [7]:
imps = dict()
cols = list(X_trn.columns)
for col in cols:
    imp_value = X_trn[col].value_counts().index[0]
    imps[col] = imp_value
    X_trn.loc[:,col] = X_trn[col].fillna(imp_value)

Standardize the training part (not implemented here, just extract the means and standard deviations, will be implemented in training process)

In [ ]:
scalers = dict()
cols = list(X_trn.columns)
for col in cols:
    avg = X_trn[col].mean()
    std = X_trn[col].std()
    scalers[col] = (avg, std)

Store the imputing and standardization information.

In [13]:
import pickle

with open('imps', 'wb') as fp:
    pickle.dump(imps, fp)
with open('scalers', 'wb') as fp:
    pickle.dump(scalers, fp) 
with open('features', 'wb') as fp:
    pickle.dump(list(X_trn.columns), fp)

In [12]:
np.save('X_trn_3d.npy', X_trn_3d)

In [14]:
imps = pickle.load(open('imps', "rb"))
scalers = pickle.load(open('scalers', "rb"))
features = pickle.load(open('features', "rb"))

Use the stored imputing information to impute missings in test set.

In [6]:
for k, v in imps.items():
    X_tst.loc[:,k] = X_tst[k].fillna(v)

Reshape the data to a 3D format. (household x months x features)

In [10]:
X_trn_3d = X_trn.as_matrix().reshape(X_trn.shape[0]/12,12,-1)
X_tst_3d = X_tst.as_matrix().reshape(X_tst.shape[0]/12,12,-1)

/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


In [13]:
np.save('X_trn_3d.npy', X_trn_3d)
np.save('X_tst_3d.npy', X_tst_3d)

Get the labels and store them.

In [16]:
y_trn = pd.read_csv(label_name+'/data_q3_y_trn.csv')
y_trn = y_trn.groupby('rlb_location_key')[label_name].max().as_matrix()
np.save('y_trn.npy', y_trn)

In [30]:
y_tst = pd.read_csv(label_name+'/data_q3_y_tst.csv')
y_tst = y_tst.groupby('rlb_location_key')[label_name].max().as_matrix()
np.save('y_tst.npy', y_tst)

In [31]:
sum(y_trn), sum(y_tst)

(223, 110)

## Begin modeling

In [3]:
scalers = pickle.load(open('scalers', "rb"))
features = pickle.load(open('features', "rb"))

In [ ]:
X_trn_3d = np.load('X_trn_3d.npy')
y_trn = np.load('y_trn.npy')

In [5]:
X_tst_3d = np.load('X_tst_3d.npy')
y_tst = np.load('y_tst.npy')

In [6]:
X_trn_3d.shape

(220000, 12, 326)

In [7]:
means = [scalers[fea][0] for fea in features]
stds = [scalers[fea][1] for fea in features]

Replace 0 standard deviations with 1. Then standardize the test set.

In [8]:
stds = list(map(lambda x:x if x!=0 else 1, stds))

In [9]:
X_tst_std = (X_tst_3d-means)/stds
del X_tst_3d

A batch generator function for feeding into batches of data.

In [11]:
def batch_generator(batch_size=64):
    means = [scalers[fea][0] for fea in features]
    stds = [scalers[fea][1] for fea in features]
    stds = list(map(lambda x:x if x!=0 else 1, stds))
    index = list(range(X_trn_3d.shape[0]))
    nb_batch = int(X_trn_3d.shape[0]/batch_size)
    while True:
        np.random.shuffle(index)
        for i in range(nb_batch):
            ind = index[i*batch_size:(i+1)*batch_size]
            yield (X_trn_3d[ind]-means)/stds, y_trn[ind]
#         if nb_batch*batch_size < X_trn_3d.shape[0]:
#             ind = index[(nb_batch*batch_size):]
#             yield (X_trn_3d[ind] - means)/stds, y_trn[ind]

A batch generator function with balanced batch samples.

In [12]:
def batch_generator_balanced(batch_size=64):
    means = [scalers[fea][0] for fea in features]
    stds = [scalers[fea][1] for fea in features]
    stds = list(map(lambda x:x if x!=0 else 1, stds))
    index = list(range(X_trn_3d.shape[0]))
    pos_ind = list(np.where(y_trn == 1)[0])
    neg_ind = list(set(index)-set(pos_ind))
    nb_batch = int(len(neg_ind)/batch_size)
    nb_batch_pos = int(len(pos_ind)/batch_size)
    half_batch_size = int(batch_size/2)
    while True:
        np.random.shuffle(neg_ind)
        for i in range(nb_batch):
            neg_batch_ind = neg_ind[i*half_batch_size:(i+1)*half_batch_size]
            j = i%nb_batch_pos
            pos_batch_ind = pos_ind[j*half_batch_size:(j+1)*half_batch_size]
            if j == nb_batch_pos-1:
                np.random.shuffle(pos_ind)
            yield (X_trn_3d[neg_batch_ind+pos_batch_ind]-means)/stds, y_trn[neg_batch_ind+pos_batch_ind]
#         if nb_batch*batch_size < X_trn_3d.shape[0]:
#             ind = index[(nb_batch*batch_size):]
#             yield (X_trn_3d[ind] - means)/stds, y_trn[ind]

In [13]:
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.models import Sequential
from keras.layers import Activation, SimpleRNN, Dense, Embedding, Convolution2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam

Using TensorFlow backend.


## RNN

In [52]:
model=Sequential([
        SimpleRNN(256, input_shape=(12, 326),
                  activation='relu', recurrent_initializer="identity"),
        BatchNormalization(),
        Dense(1, activation='sigmoid')
    ])
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [53]:
nb_sample = X_trn_3d.shape[0]
model.fit_generator(batch_generator_balanced(), steps_per_epoch=int(nb_sample/64), epochs=5)#, class_weight={0:1,1:1000})
pred = model.predict(X_tst_std)
roc_auc_score(y_tst, pred)

Epoch 1/5
3437/3437 [==============================] - 82s - loss: 0.0385 - acc: 0.9882    
Epoch 2/5
3437/3437 [==============================] - 82s - loss: 0.0201 - acc: 0.9948    
Epoch 3/5
3437/3437 [==============================] - 85s - loss: 0.0302 - acc: 0.9966    
Epoch 4/5
3437/3437 [==============================] - 84s - loss: 0.0621 - acc: 0.9940    
Epoch 5/5
3437/3437 [==============================] - 82s - loss: 0.0483 - acc: 0.9959    


0.50843531889905258

In [54]:
model.fit_generator(batch_generator_balanced(), steps_per_epoch=int(nb_sample/64), epochs=3)
pred = model.predict(X_tst_std)
roc_auc_score(y_tst, pred)

Epoch 1/3
3437/3437 [==============================] - 81s - loss: 0.0577 - acc: 0.9957    
Epoch 2/3
3437/3437 [==============================] - 85s - loss: 0.0600 - acc: 0.9956    
Epoch 3/3
3437/3437 [==============================] - 84s - loss: 0.0408 - acc: 0.9969    


0.5187617895703508

## CNN

In [14]:
X_trn_3d = X_trn_3d.reshape(-1, 1, 12, 326)
X_tst_std = X_tst_std.reshape(-1, 1, 12, 326)

In [15]:
X_trn_3d.shape, X_tst_std.shape

((220000, 1, 12, 326), (93461, 1, 12, 326))

In [16]:
model = Sequential()

model.add(Convolution2D(
    input_shape=(1, 12, 326),
    filters=32,
    kernel_size=(1,4),
    strides=1,
    padding='same',
    data_format='channels_first',
))
model.add(Activation('relu'))


model.add(MaxPooling2D(
    pool_size=(1,2),
    strides=2,
    padding='same',
    data_format='channels_first',
))


model.add(Convolution2D(64, (1,2), strides=1, padding='same', data_format='channels_first'))
model.add(Activation('relu'))

model.add(MaxPooling2D((1,2), 2, 'same', data_format='channels_first'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [17]:
model.compile(Adam(), 'binary_crossentropy', ['accuracy'])

In [19]:
nb_sample = X_trn_3d.shape[0]
model.fit_generator(batch_generator_balanced(), steps_per_epoch=int(nb_sample/64), epochs=1)
pred = model.predict(X_tst_std)
roc_auc_score(y_tst, pred)

Epoch 1/1
3437/3437 [==============================] - 2105s - loss: 0.0655 - acc: 0.9758  


0.52594109621458018

In [ ]:
model.fit_generator(batch_generator_balanced(), steps_per_epoch=int(nb_sample/64), epochs=1)
pred = model.predict(X_tst_std)
roc_auc_score(y_tst, pred)

Epoch 1/1
3017/3437 [=========================>....] - ETA: 310s - loss: 0.0101 - acc: 0.9970

In [15]:
! ls mt_hh/data_q3* -la

-rw-r--r-- 1 pl74056 users 1205445583 Aug  4 02:50 mt_hh/data_q3_X_1.csv
-rw-r--r-- 1 pl74056 users 1213549121 Aug  4 03:01 mt_hh/data_q3_X_2.csv
-rw-r--r-- 1 pl74056 users 1214585416 Aug  4 03:12 mt_hh/data_q3_X_3.csv
-rw-r--r-- 1 pl74056 users 1220695224 Aug  4 03:24 mt_hh/data_q3_X_4.csv
-rw-r--r-- 1 pl74056 users 2339155925 Aug  8 20:05 mt_hh/data_q3_X_trn.csv
-rw-r--r-- 1 pl74056 users  990931199 Aug  8 20:10 mt_hh/data_q3_X_tst.csv
-rw-r--r-- 1 pl74056 users    7746817 Aug  8 04:52 mt_hh/data_q3_y.csv
-rw-r--r-- 1 pl74056 users   58080023 Aug  8 20:06 mt_hh/data_q3_y_trn.csv
-rw-r--r-- 1 pl74056 users   24673727 Aug  8 20:06 mt_hh/data_q3_y_tst.csv
